In [7]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, make_scorer, f1_score
from sklearn.utils.class_weight import compute_class_weight

# Load data
df = pd.read_csv('fight_data.csv')

# Feature and label setup
input_features = [
    'timer', 'has_round_started', 'is_round_over',
    'health', 'x_coord', 'y_coord', 'is_jumping', 'is_crouching', 'is_player_in_move', 'move_id',
    'Player2 health', 'Player2 x_coord', 'Player2 y_coord',
    'Player2 is_jumping', 'Player2 is_crouching', 'Player2 is_player_in_move', 'Player2 move_id'
]

label_columns = [
    'player1_buttons_Up', 'player1_buttons_Down', 'player1_buttons_Right', 'player1_buttons_Left',
    'player1_buttons_Y', 'player1_buttons_B', 'player1_buttons_A',
    'player1_buttons_X', 'player1_buttons_L', 'player1_buttons_R'
]

# Offensive buttons
offensive_buttons = [
    'player1_buttons_Y', 'player1_buttons_B', 'player1_buttons_A',
    'player1_buttons_X', 'player1_buttons_L', 'player1_buttons_R'
]

# Clean labels
df[label_columns] = df[label_columns].astype(int)
X = df[input_features]
y = df[label_columns]

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Compute class weights and inflate offensive moves
class_weight_dict = {}
for col in label_columns:
    classes = np.array([0, 1])
    if len(np.unique(y_train[col])) < 2:
        weights = [1.0, 1.0]
    else:
        weights = compute_class_weight('balanced', classes=classes, y=y_train[col])
        weights = list(weights)

    # Boost offensive '1' label
    if col in offensive_buttons:
        weights[1] *= 2.5

    class_weight_dict[col] = {0: weights[0], 1: weights[1]}

# Custom scoring function to weight offensive buttons more
def offensive_weighted_f1(y_true, y_pred):
    scores = []
    for i, col in enumerate(label_columns):
        weight = 2.0 if col in offensive_buttons else 1.0
        f1 = f1_score(y_true[:, i], y_pred[:, i], zero_division=1)
        scores.append(f1 * weight)
    return np.mean(scores)

custom_scorer = make_scorer(offensive_weighted_f1)

# Grid Search parameters
param_grid = {
    'n_estimators': [100],
    'max_depth': [20],
    'min_samples_split': [2],
    'min_samples_leaf': [1]
}

# Train one tuned model per label
best_models = []
for col in label_columns:
    print(f"Training model for {col}")
    clf = RandomForestClassifier(random_state=42, class_weight=class_weight_dict[col])
    grid_search = GridSearchCV(clf, param_grid, cv=3, scoring='f1_macro', n_jobs=-1)
    grid_search.fit(X_train, y_train[col])
    best_models.append(grid_search.best_estimator_)

# Combine predictions from all models
y_pred_combined = np.column_stack([model.predict(X_test) for model in best_models])

# Evaluate
print("\nOverall Evaluation with Offensive Bias (Macro-Average):")
print(classification_report(y_test, y_pred_combined, target_names=label_columns))
   

Training model for player1_buttons_Up
Training model for player1_buttons_Down
Training model for player1_buttons_Right
Training model for player1_buttons_Left
Training model for player1_buttons_Y
Training model for player1_buttons_B
Training model for player1_buttons_A
Training model for player1_buttons_X
Training model for player1_buttons_L
Training model for player1_buttons_R

Overall Evaluation with Offensive Bias (Macro-Average):
                       precision    recall  f1-score   support

   player1_buttons_Up       0.06      0.22      0.09       266
 player1_buttons_Down       0.26      0.34      0.30      1697
player1_buttons_Right       0.21      0.39      0.27      1285
 player1_buttons_Left       0.17      0.24      0.20      1180
    player1_buttons_Y       0.07      0.46      0.12       491
    player1_buttons_B       0.06      0.47      0.10       266
    player1_buttons_A       0.00      0.00      0.00         0
    player1_buttons_X       0.00      0.00      0.00     

C:\Users\riyya\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\riyya\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\riyya\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\riyya\anaconda3\Lib\site-package

In [8]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier
import joblib

# Assume X_train and y_train are already prepared
base_model = RandomForestClassifier(n_estimators=10, max_depth=20, class_weight="balanced", random_state=42)
multi_model = MultiOutputClassifier(base_model)
multi_model.fit(X_train, y_train)

joblib.dump(multi_model, 'trained_fight_model.pkl')


['trained_fight_model.pkl']

In [9]:
from sklearn.neural_network import MLPClassifier
from sklearn.multioutput import MultiOutputClassifier
import joblib

# Define MLP base model
mlp_base_model = MLPClassifier(
    hidden_layer_sizes=(100, 50),  # You can tune this
    activation='relu',
    solver='adam',
    max_iter=300,
    random_state=42
)

# Wrap in MultiOutputClassifier for multilabel classification
mlp_multi_model = MultiOutputClassifier(mlp_base_model)

# Train the model
mlp_multi_model.fit(X_train, y_train)

# Save the model
joblib.dump(mlp_multi_model, 'trained_fight_model_mlp.pkl')


['trained_fight_model_mlp.pkl']